In [1]:
import tensorflow as tf
print(tf.__version__)

2.11.0


In [ ]:
!pip install tensorflow==2.11.0

In [2]:
!pip install keras-cv --upgrade --quiet
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcudnn8 is already the newest version (8.1.0.77-1+cuda11.2).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [3]:
import keras_cv
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
from PIL import Image
from IPython.display import Image as IImage
from google.colab import files
import pandas as pd

# Enable mixed precision
# (only do this if you have a recent NVIDIA GPU)
keras.mixed_precision.set_global_policy("mixed_float16")

# Instantiate the Stable Diffusion model
model = keras_cv.models.StableDiffusion(jit_compile=True, img_width=640)

You do not have Waymo Open Dataset installed, so KerasCV Waymo metrics are not available.
By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE


In [4]:
drive_path = '/content/drive/MyDrive/Year 4 (2022 - 2023)/Spring 2023/Art Intel/Project 3'

In [5]:
seed = 12345
noise = tf.random.normal((512 // 8, 640 // 8, 4), seed=seed)

In [6]:
def export_as_gif(filename, images, frames_per_second=10, rubber_band=False):
    if rubber_band:
        images += images[2:-1][::-1]
    images[0].save(
        filename,
        save_all=True,
        append_images=images[1:],
        duration=1000 // frames_per_second,
        loop=0,
    )

def plot_images(images):
    plt.figure(figsize=(20, 20))
    for i in range(len(images)):
        ax = plt.subplot(1, len(images), i + 1)
        plt.imshow(images[i])
        plt.axis("off")

In [7]:
prompts_df = pd.read_csv(f'{drive_path}/prompts.csv')
print(prompts_df.shape)
prompts_df.head()

(81, 3)


,Section,Scene,Prompts
0,Opening montage,"The camera pans over a deserted cityscape, wit...","Desolate cityscape, post-apocalyptic, towering..."
1,NaN,NaN,"Close-up of a swaying traffic light, rusted me..."
2,NaN,NaN,"Shattered storefront window, broken glass, deb..."
3,NaN,NaN,"Rusted bicycle, abandoned, stillness in the ai..."
4,NaN,NaN,"Graffiti-covered wall, faded paint, stark cont..."


In [8]:
# prompt_1 = "A watercolor painting of a Golden Retriever at the beach"
# prompt_2 = "A still life DSLR photo of a bowl of fruit"
# interpolation_steps = 5

# encoding_1 = tf.squeeze(model.encode_text(prompt_1))
# encoding_2 = tf.squeeze(model.encode_text(prompt_2))

# interpolated_encodings = tf.linspace(encoding_1, encoding_2, interpolation_steps)

# # Show the size of the latent manifold
# print(f"Encoding shape: {encoding_1.shape}")

In [9]:
duration = 2 * 60 + 40
fps = 5
total_frames = duration * fps
total_frames

800

In [10]:
prompts_all = prompts_df['Prompts'].to_list()
prompt_encodings = []
for prompt in prompts_all:
  encoding = tf.squeeze(model.encode_text(prompt))
  prompt_encodings.append(encoding)

In [11]:
# prompt_encodings[0].numpy().shape
# prompt_encodings[0].numpy().flatten().reshape(77, 768) == prompt_encodings[0].numpy()
# flat_arrs = [enc.numpy().flatten() for enc in prompt_encodings]
# flat_encodings = np.vstack(flat_arrs)
# len(flat_encodings), len(flat_encodings[0])
# np.savetxt(f"{drive_path}/flat_encodings.csv", flat_encodings, delimiter=",")

# tmp0 = tf.linspace(prompt_encodings[0], prompt_encodings[1], 7)
# tmp1 = tf.linspace(prompt_encodings[1], prompt_encodings[2], 7)
# tmp_concat = tf.concat([tmp0, tmp1], axis=0)
# tmp0.shape, tmp1.shape, tmp_concat.shape

In [12]:
interpolation_steps = 10
all_interpolated_encodings = []
for i in range(len(prompts_all)-1):
  interpolated_encodings = tf.linspace(prompt_encodings[i], prompt_encodings[i+1], interpolation_steps)
  all_interpolated_encodings.append(interpolated_encodings)
interpolated_encodings_stack = tf.concat(all_interpolated_encodings, axis=0)

In [13]:
interpolated_encodings_stack.shape

TensorShape([800, 77, 768])

In [14]:
num_steps = 20
batch_size = 5
batches = interpolated_encodings_stack.shape[0] // batch_size
batched_encodings = tf.split(interpolated_encodings_stack, batches)
batches

160

In [15]:
images = []

In [ ]:
# 0 - 9
for batch in range(10):
  offset = 0
  batch = batch + offset * 10
  print(f'\nBatch: {batch}')
  imgs = model.generate_image(
            batched_encodings[batch],
            batch_size=batch_size,
            num_steps=num_steps,
            diffusion_noise=noise,
        )
  for idx, img in enumerate(imgs):
    img_idx = batch * batch_size + idx
    print(f'Saving image: {img_idx}')
    img_image = Image.fromarray(img)
    images.append(img_image)
    img_image.save(f'{drive_path}/Frames/img_{img_idx}.png')
    img_image.save(f'img_{img_idx}.png')
    files.download(f'img_{img_idx}.png')


Batch: 0


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


KeyboardInterrupt: ignored

In [ ]:
# 10 - 19
for batch in range(10):
  offset = 1
  batch = batch + offset * 10
  print(f'\nBatch: {batch}')
  imgs = model.generate_image(
            batched_encodings[batch],
            batch_size=batch_size,
            num_steps=num_steps,
            diffusion_noise=noise,
        )
  for idx, img in enumerate(imgs):
    img_idx = batch * batch_size + idx
    print(f'Saving image: {img_idx}')
    img_image = Image.fromarray(img)
    images.append(img_image)
    img_image.save(f'{drive_path}/Frames/img_{img_idx}.png')
    img_image.save(f'img_{img_idx}.png')
    # files.download(f'img_{img_idx}.png')


Batch: 10
20/20 [==============================] - 28s 1s/step
Saving image: 50
Saving image: 51
Saving image: 52
Saving image: 53
Saving image: 54

Batch: 11
20/20 [==============================] - 28s 1s/step
Saving image: 55
Saving image: 56
Saving image: 57
Saving image: 58
Saving image: 59

Batch: 12
20/20 [==============================] - 28s 1s/step
Saving image: 60
Saving image: 61
Saving image: 62
Saving image: 63
Saving image: 64

Batch: 13
20/20 [==============================] - 28s 1s/step
Saving image: 65
Saving image: 66
Saving image: 67
Saving image: 68
Saving image: 69

Batch: 14
20/20 [==============================] - 29s 1s/step
Saving image: 70
Saving image: 71
Saving image: 72
Saving image: 73
Saving image: 74

Batch: 15
20/20 [==============================] - 27s 1s/step
Saving image: 75
Saving image: 76
Saving image: 77
Saving image: 78
Saving image: 79

Batch: 16
20/20 [==============================] - 27s 1s/step
Saving image: 80
Saving image: 81
Saving i

In [ ]:
# 20 - 29
for batch in range(10):
  offset = 2
  batch = batch + offset * 10
  print(f'\nBatch: {batch}')
  imgs = model.generate_image(
            batched_encodings[batch],
            batch_size=batch_size,
            num_steps=num_steps,
            diffusion_noise=noise,
        )
  for idx, img in enumerate(imgs):
    img_idx = batch * batch_size + idx
    print(f'Saving image: {img_idx}')
    img_image = Image.fromarray(img)
    images.append(img_image)
    img_image.save(f'{drive_path}/Frames/img_{img_idx}.png')
    img_image.save(f'img_{img_idx}.png')
    # files.download(f'img_{img_idx}.png')


Batch: 20
20/20 [==============================] - 29s 1s/step
Saving image: 100
Saving image: 101
Saving image: 102
Saving image: 103
Saving image: 104

Batch: 21
20/20 [==============================] - 29s 1s/step
Saving image: 105
Saving image: 106
Saving image: 107
Saving image: 108
Saving image: 109

Batch: 22
20/20 [==============================] - 29s 1s/step
Saving image: 110
Saving image: 111
Saving image: 112
Saving image: 113
Saving image: 114

Batch: 23
20/20 [==============================] - 28s 1s/step
Saving image: 115
Saving image: 116
Saving image: 117
Saving image: 118
Saving image: 119

Batch: 24
20/20 [==============================] - 30s 2s/step
Saving image: 120
Saving image: 121
Saving image: 122
Saving image: 123
Saving image: 124

Batch: 25
20/20 [==============================] - 29s 1s/step
Saving image: 125
Saving image: 126
Saving image: 127
Saving image: 128
Saving image: 129

Batch: 26
20/20 [==============================] - 29s 1s/step
Saving image

In [ ]:
# 30 - 39
for batch in range(10):
  offset = 3
  batch = batch + offset * 10
  print(f'\nBatch: {batch}')
  imgs = model.generate_image(
            batched_encodings[batch],
            batch_size=batch_size,
            num_steps=num_steps,
            diffusion_noise=noise,
        )
  for idx, img in enumerate(imgs):
    img_idx = batch * batch_size + idx
    print(f'Saving image: {img_idx}')
    img_image = Image.fromarray(img)
    images.append(img_image)
    img_image.save(f'{drive_path}/Frames/img_{img_idx}.png')
    img_image.save(f'img_{img_idx}.png')
    # files.download(f'img_{img_idx}.png')


Batch: 30
20/20 [==============================] - 29s 1s/step
Saving image: 150
Saving image: 151
Saving image: 152
Saving image: 153
Saving image: 154

Batch: 31
20/20 [==============================] - 30s 1s/step
Saving image: 155
Saving image: 156
Saving image: 157
Saving image: 158
Saving image: 159

Batch: 32
20/20 [==============================] - 29s 1s/step
Saving image: 160
Saving image: 161
Saving image: 162
Saving image: 163
Saving image: 164

Batch: 33
20/20 [==============================] - 29s 1s/step
Saving image: 165
Saving image: 166
Saving image: 167
Saving image: 168
Saving image: 169

Batch: 34
20/20 [==============================] - 29s 1s/step
Saving image: 170
Saving image: 171
Saving image: 172
Saving image: 173
Saving image: 174

Batch: 35
20/20 [==============================] - 29s 1s/step
Saving image: 175
Saving image: 176
Saving image: 177
Saving image: 178
Saving image: 179

Batch: 36
20/20 [==============================] - 29s 1s/step
Saving image

In [ ]:
# 40 - 49
for batch in range(10):
  offset = 4
  batch = batch + offset * 10
  print(f'\nBatch: {batch}')
  imgs = model.generate_image(
            batched_encodings[batch],
            batch_size=batch_size,
            num_steps=num_steps,
            diffusion_noise=noise,
        )
  for idx, img in enumerate(imgs):
    img_idx = batch * batch_size + idx
    print(f'Saving image: {img_idx}')
    img_image = Image.fromarray(img)
    images.append(img_image)
    img_image.save(f'{drive_path}/Frames/img_{img_idx}.png')
    img_image.save(f'img_{img_idx}.png')
    # files.download(f'img_{img_idx}.png')


Batch: 40
20/20 [==============================] - 112s 1s/step
Saving image: 200
Saving image: 201
Saving image: 202
Saving image: 203
Saving image: 204

Batch: 41
20/20 [==============================] - 26s 1s/step
Saving image: 205
Saving image: 206
Saving image: 207
Saving image: 208
Saving image: 209

Batch: 42
20/20 [==============================] - 26s 1s/step
Saving image: 210
Saving image: 211
Saving image: 212
Saving image: 213
Saving image: 214

Batch: 43
20/20 [==============================] - 27s 1s/step
Saving image: 215
Saving image: 216
Saving image: 217
Saving image: 218
Saving image: 219

Batch: 44
20/20 [==============================] - 28s 1s/step
Saving image: 220
Saving image: 221
Saving image: 222
Saving image: 223
Saving image: 224

Batch: 45
20/20 [==============================] - 28s 1s/step
Saving image: 225
Saving image: 226
Saving image: 227
Saving image: 228
Saving image: 229

Batch: 46
20/20 [==============================] - 27s 1s/step
Saving imag

In [ ]:
# 50 - 59
for batch in range(10):
  offset = 5
  batch = batch + offset * 10
  print(f'\nBatch: {batch}')
  imgs = model.generate_image(
            batched_encodings[batch],
            batch_size=batch_size,
            num_steps=num_steps,
            diffusion_noise=noise,
        )
  for idx, img in enumerate(imgs):
    img_idx = batch * batch_size + idx
    print(f'Saving image: {img_idx}')
    img_image = Image.fromarray(img)
    images.append(img_image)
    img_image.save(f'{drive_path}/Frames/img_{img_idx}.png')
    img_image.save(f'img_{img_idx}.png')
    # files.download(f'img_{img_idx}.png')


Batch: 50
20/20 [==============================] - 27s 1s/step
Saving image: 250
Saving image: 251
Saving image: 252
Saving image: 253
Saving image: 254

Batch: 51
20/20 [==============================] - 27s 1s/step
Saving image: 255
Saving image: 256
Saving image: 257
Saving image: 258
Saving image: 259

Batch: 52
20/20 [==============================] - 28s 1s/step
Saving image: 260
Saving image: 261
Saving image: 262
Saving image: 263
Saving image: 264

Batch: 53
20/20 [==============================] - 29s 1s/step
Saving image: 265
Saving image: 266
Saving image: 267
Saving image: 268
Saving image: 269

Batch: 54
20/20 [==============================] - 30s 1s/step
Saving image: 270
Saving image: 271
Saving image: 272
Saving image: 273
Saving image: 274

Batch: 55
20/20 [==============================] - 29s 1s/step
Saving image: 275
Saving image: 276
Saving image: 277
Saving image: 278
Saving image: 279

Batch: 56
20/20 [==============================] - 27s 1s/step
Saving image

In [ ]:
# 60 - 69
for batch in range(10):
  offset = 6
  batch = batch + offset * 10
  print(f'\nBatch: {batch}')
  imgs = model.generate_image(
            batched_encodings[batch],
            batch_size=batch_size,
            num_steps=num_steps,
            diffusion_noise=noise,
        )
  for idx, img in enumerate(imgs):
    img_idx = batch * batch_size + idx
    print(f'Saving image: {img_idx}')
    img_image = Image.fromarray(img)
    images.append(img_image)
    img_image.save(f'{drive_path}/Frames/img_{img_idx}.png')
    img_image.save(f'img_{img_idx}.png')
    # files.download(f'img_{img_idx}.png')


Batch: 60
20/20 [==============================] - 29s 1s/step
Saving image: 300
Saving image: 301
Saving image: 302
Saving image: 303
Saving image: 304

Batch: 61
20/20 [==============================] - 29s 1s/step
Saving image: 305
Saving image: 306
Saving image: 307
Saving image: 308
Saving image: 309

Batch: 62
20/20 [==============================] - 28s 1s/step
Saving image: 310
Saving image: 311
Saving image: 312
Saving image: 313
Saving image: 314

Batch: 63
20/20 [==============================] - 29s 1s/step
Saving image: 315
Saving image: 316
Saving image: 317
Saving image: 318
Saving image: 319

Batch: 64
20/20 [==============================] - 29s 1s/step
Saving image: 320
Saving image: 321
Saving image: 322
Saving image: 323
Saving image: 324

Batch: 65
20/20 [==============================] - 29s 1s/step
Saving image: 325
Saving image: 326
Saving image: 327
Saving image: 328
Saving image: 329

Batch: 66
20/20 [==============================] - 28s 1s/step
Saving image

In [16]:
# 70 - 79
for batch in range(10):
  offset = 7
  batch = batch + offset * 10
  print(f'\nBatch: {batch}')
  imgs = model.generate_image(
            batched_encodings[batch],
            batch_size=batch_size,
            num_steps=num_steps,
            diffusion_noise=noise,
        )
  for idx, img in enumerate(imgs):
    img_idx = batch * batch_size + idx
    print(f'Saving image: {img_idx}')
    img_image = Image.fromarray(img)
    images.append(img_image)
    img_image.save(f'{drive_path}/Frames/img_{img_idx}.png')
    img_image.save(f'img_{img_idx}.png')
    # files.download(f'img_{img_idx}.png')


Batch: 70


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


20/20 [==============================] - 115s 1s/step
Saving image: 350
Saving image: 351
Saving image: 352
Saving image: 353
Saving image: 354

Batch: 71
20/20 [==============================] - 29s 1s/step
Saving image: 355
Saving image: 356
Saving image: 357
Saving image: 358
Saving image: 359

Batch: 72
20/20 [==============================] - 29s 1s/step
Saving image: 360
Saving image: 361
Saving image: 362
Saving image: 363
Saving image: 364

Batch: 73
20/20 [==============================] - 28s 1s/step
Saving image: 365
Saving image: 366
Saving image: 367
Saving image: 368
Saving image: 369

Batch: 74
20/20 [==============================] - 28s 1s/step
Saving image: 370
Saving image: 371
Saving image: 372
Saving image: 373
Saving image: 374

Batch: 75
20/20 [==============================] - 28s 1s/step
Saving image: 375
Saving image: 376
Saving image: 377
Saving image: 378
Saving image: 379

Batch: 76
20/20 [==============================] - 29s 1s/step
Saving image: 380
Savi

In [17]:
# 80 - 89
for batch in range(10):
  offset = 8
  batch = batch + offset * 10
  print(f'\nBatch: {batch}')
  imgs = model.generate_image(
            batched_encodings[batch],
            batch_size=batch_size,
            num_steps=num_steps,
            diffusion_noise=noise,
        )
  for idx, img in enumerate(imgs):
    img_idx = batch * batch_size + idx
    print(f'Saving image: {img_idx}')
    img_image = Image.fromarray(img)
    images.append(img_image)
    img_image.save(f'{drive_path}/Frames/img_{img_idx}.png')
    img_image.save(f'img_{img_idx}.png')
    # files.download(f'img_{img_idx}.png')


Batch: 80
20/20 [==============================] - 28s 1s/step
Saving image: 400
Saving image: 401
Saving image: 402
Saving image: 403
Saving image: 404

Batch: 81
20/20 [==============================] - 29s 1s/step
Saving image: 405
Saving image: 406
Saving image: 407
Saving image: 408
Saving image: 409

Batch: 82
20/20 [==============================] - 27s 1s/step
Saving image: 410
Saving image: 411
Saving image: 412
Saving image: 413
Saving image: 414

Batch: 83
20/20 [==============================] - 30s 1s/step
Saving image: 415
Saving image: 416
Saving image: 417
Saving image: 418
Saving image: 419

Batch: 84
20/20 [==============================] - 29s 1s/step
Saving image: 420
Saving image: 421
Saving image: 422
Saving image: 423
Saving image: 424

Batch: 85
20/20 [==============================] - 28s 1s/step
Saving image: 425
Saving image: 426
Saving image: 427
Saving image: 428
Saving image: 429

Batch: 86
20/20 [==============================] - 29s 1s/step
Saving image

In [18]:
# 90 - 99
for batch in range(10):
  offset = 9
  batch = batch + offset * 10
  print(f'\nBatch: {batch}')
  imgs = model.generate_image(
            batched_encodings[batch],
            batch_size=batch_size,
            num_steps=num_steps,
            diffusion_noise=noise,
        )
  for idx, img in enumerate(imgs):
    img_idx = batch * batch_size + idx
    print(f'Saving image: {img_idx}')
    img_image = Image.fromarray(img)
    images.append(img_image)
    img_image.save(f'{drive_path}/Frames/img_{img_idx}.png')
    img_image.save(f'img_{img_idx}.png')
    # files.download(f'img_{img_idx}.png')


Batch: 90
20/20 [==============================] - 29s 1s/step
Saving image: 450
Saving image: 451
Saving image: 452
Saving image: 453
Saving image: 454

Batch: 91
20/20 [==============================] - 29s 1s/step
Saving image: 455
Saving image: 456
Saving image: 457
Saving image: 458
Saving image: 459

Batch: 92
20/20 [==============================] - 29s 1s/step
Saving image: 460
Saving image: 461
Saving image: 462
Saving image: 463
Saving image: 464

Batch: 93
20/20 [==============================] - 28s 1s/step
Saving image: 465
Saving image: 466
Saving image: 467
Saving image: 468
Saving image: 469

Batch: 94
20/20 [==============================] - 28s 1s/step
Saving image: 470
Saving image: 471
Saving image: 472
Saving image: 473
Saving image: 474

Batch: 95
20/20 [==============================] - 29s 1s/step
Saving image: 475
Saving image: 476
Saving image: 477
Saving image: 478
Saving image: 479

Batch: 96
20/20 [==============================] - 28s 1s/step
Saving image

In [19]:
# 100 - 109
for batch in range(10):
  offset = 10
  batch = batch + offset * 10
  print(f'\nBatch: {batch}')
  imgs = model.generate_image(
            batched_encodings[batch],
            batch_size=batch_size,
            num_steps=num_steps,
            diffusion_noise=noise,
        )
  for idx, img in enumerate(imgs):
    img_idx = batch * batch_size + idx
    print(f'Saving image: {img_idx}')
    img_image = Image.fromarray(img)
    images.append(img_image)
    img_image.save(f'{drive_path}/Frames/img_{img_idx}.png')
    img_image.save(f'img_{img_idx}.png')
    # files.download(f'img_{img_idx}.png')


Batch: 100
20/20 [==============================] - 29s 1s/step
Saving image: 500
Saving image: 501
Saving image: 502
Saving image: 503
Saving image: 504

Batch: 101
20/20 [==============================] - 30s 2s/step
Saving image: 505
Saving image: 506
Saving image: 507
Saving image: 508
Saving image: 509

Batch: 102
20/20 [==============================] - 29s 1s/step
Saving image: 510
Saving image: 511
Saving image: 512
Saving image: 513
Saving image: 514

Batch: 103
20/20 [==============================] - 29s 1s/step
Saving image: 515
Saving image: 516
Saving image: 517
Saving image: 518
Saving image: 519

Batch: 104
20/20 [==============================] - 28s 1s/step
Saving image: 520
Saving image: 521
Saving image: 522
Saving image: 523
Saving image: 524

Batch: 105
20/20 [==============================] - 29s 1s/step
Saving image: 525
Saving image: 526
Saving image: 527
Saving image: 528
Saving image: 529

Batch: 106
20/20 [==============================] - 27s 1s/step
Savin

In [20]:
# 110 - 119
for batch in range(10):
  offset = 11
  batch = batch + offset * 10
  print(f'\nBatch: {batch}')
  imgs = model.generate_image(
            batched_encodings[batch],
            batch_size=batch_size,
            num_steps=num_steps,
            diffusion_noise=noise,
        )
  for idx, img in enumerate(imgs):
    img_idx = batch * batch_size + idx
    print(f'Saving image: {img_idx}')
    img_image = Image.fromarray(img)
    images.append(img_image)
    img_image.save(f'{drive_path}/Frames/img_{img_idx}.png')
    img_image.save(f'img_{img_idx}.png')
    # files.download(f'img_{img_idx}.png')


Batch: 110
20/20 [==============================] - 28s 1s/step
Saving image: 550
Saving image: 551
Saving image: 552
Saving image: 553
Saving image: 554

Batch: 111
20/20 [==============================] - 29s 1s/step
Saving image: 555
Saving image: 556
Saving image: 557
Saving image: 558
Saving image: 559

Batch: 112
20/20 [==============================] - 28s 1s/step
Saving image: 560
Saving image: 561
Saving image: 562
Saving image: 563
Saving image: 564

Batch: 113
20/20 [==============================] - 29s 1s/step
Saving image: 565
Saving image: 566
Saving image: 567
Saving image: 568
Saving image: 569

Batch: 114
20/20 [==============================] - 29s 1s/step
Saving image: 570
Saving image: 571
Saving image: 572
Saving image: 573
Saving image: 574

Batch: 115
20/20 [==============================] - 29s 1s/step
Saving image: 575
Saving image: 576
Saving image: 577
Saving image: 578
Saving image: 579

Batch: 116
20/20 [==============================] - 30s 2s/step
Savin

In [21]:
# 120 - 129
for batch in range(10):
  offset = 12
  batch = batch + offset * 10
  print(f'\nBatch: {batch}')
  imgs = model.generate_image(
            batched_encodings[batch],
            batch_size=batch_size,
            num_steps=num_steps,
            diffusion_noise=noise,
        )
  for idx, img in enumerate(imgs):
    img_idx = batch * batch_size + idx
    print(f'Saving image: {img_idx}')
    img_image = Image.fromarray(img)
    images.append(img_image)
    img_image.save(f'{drive_path}/Frames/img_{img_idx}.png')
    img_image.save(f'img_{img_idx}.png')
    # files.download(f'img_{img_idx}.png')


Batch: 120
20/20 [==============================] - 29s 1s/step
Saving image: 600
Saving image: 601
Saving image: 602
Saving image: 603
Saving image: 604

Batch: 121
20/20 [==============================] - 27s 1s/step
Saving image: 605
Saving image: 606
Saving image: 607
Saving image: 608
Saving image: 609

Batch: 122
20/20 [==============================] - 28s 1s/step
Saving image: 610
Saving image: 611
Saving image: 612
Saving image: 613
Saving image: 614

Batch: 123
20/20 [==============================] - 29s 1s/step
Saving image: 615
Saving image: 616
Saving image: 617
Saving image: 618
Saving image: 619

Batch: 124
20/20 [==============================] - 29s 1s/step
Saving image: 620
Saving image: 621
Saving image: 622
Saving image: 623
Saving image: 624

Batch: 125
20/20 [==============================] - 29s 1s/step
Saving image: 625
Saving image: 626
Saving image: 627
Saving image: 628
Saving image: 629

Batch: 126
20/20 [==============================] - 27s 1s/step
Savin

In [22]:
# 130 - 139
for batch in range(10):
  offset = 13
  batch = batch + offset * 10
  print(f'\nBatch: {batch}')
  imgs = model.generate_image(
            batched_encodings[batch],
            batch_size=batch_size,
            num_steps=num_steps,
            diffusion_noise=noise,
        )
  for idx, img in enumerate(imgs):
    img_idx = batch * batch_size + idx
    print(f'Saving image: {img_idx}')
    img_image = Image.fromarray(img)
    images.append(img_image)
    img_image.save(f'{drive_path}/Frames/img_{img_idx}.png')
    img_image.save(f'img_{img_idx}.png')
    # files.download(f'img_{img_idx}.png')


Batch: 130
20/20 [==============================] - 28s 1s/step
Saving image: 650
Saving image: 651
Saving image: 652
Saving image: 653
Saving image: 654

Batch: 131
20/20 [==============================] - 28s 1s/step
Saving image: 655
Saving image: 656
Saving image: 657
Saving image: 658
Saving image: 659

Batch: 132
20/20 [==============================] - 28s 1s/step
Saving image: 660
Saving image: 661
Saving image: 662
Saving image: 663
Saving image: 664

Batch: 133
20/20 [==============================] - 29s 1s/step
Saving image: 665
Saving image: 666
Saving image: 667
Saving image: 668
Saving image: 669

Batch: 134
20/20 [==============================] - 28s 1s/step
Saving image: 670
Saving image: 671
Saving image: 672
Saving image: 673
Saving image: 674

Batch: 135
20/20 [==============================] - 27s 1s/step
Saving image: 675
Saving image: 676
Saving image: 677
Saving image: 678
Saving image: 679

Batch: 136
20/20 [==============================] - 29s 1s/step
Savin

In [23]:
# 140 - 149
for batch in range(10):
  offset = 14
  batch = batch + offset * 10
  print(f'\nBatch: {batch}')
  imgs = model.generate_image(
            batched_encodings[batch],
            batch_size=batch_size,
            num_steps=num_steps,
            diffusion_noise=noise,
        )
  for idx, img in enumerate(imgs):
    img_idx = batch * batch_size + idx
    print(f'Saving image: {img_idx}')
    img_image = Image.fromarray(img)
    images.append(img_image)
    img_image.save(f'{drive_path}/Frames/img_{img_idx}.png')
    img_image.save(f'img_{img_idx}.png')
    # files.download(f'img_{img_idx}.png')


Batch: 140
20/20 [==============================] - 29s 1s/step
Saving image: 700
Saving image: 701
Saving image: 702
Saving image: 703
Saving image: 704

Batch: 141
20/20 [==============================] - 27s 1s/step
Saving image: 705
Saving image: 706
Saving image: 707
Saving image: 708
Saving image: 709

Batch: 142
20/20 [==============================] - 29s 1s/step
Saving image: 710
Saving image: 711
Saving image: 712
Saving image: 713
Saving image: 714

Batch: 143
20/20 [==============================] - 29s 1s/step
Saving image: 715
Saving image: 716
Saving image: 717
Saving image: 718
Saving image: 719

Batch: 144
20/20 [==============================] - 29s 1s/step
Saving image: 720
Saving image: 721
Saving image: 722
Saving image: 723
Saving image: 724

Batch: 145
20/20 [==============================] - 30s 2s/step
Saving image: 725
Saving image: 726
Saving image: 727
Saving image: 728
Saving image: 729

Batch: 146
20/20 [==============================] - 29s 1s/step
Savin

In [24]:
# 150 - 159
for batch in range(10):
  offset = 15
  batch = batch + offset * 10
  print(f'\nBatch: {batch}')
  imgs = model.generate_image(
            batched_encodings[batch],
            batch_size=batch_size,
            num_steps=num_steps,
            diffusion_noise=noise,
        )
  for idx, img in enumerate(imgs):
    img_idx = batch * batch_size + idx
    print(f'Saving image: {img_idx}')
    img_image = Image.fromarray(img)
    images.append(img_image)
    img_image.save(f'{drive_path}/Frames/img_{img_idx}.png')
    img_image.save(f'img_{img_idx}.png')
    # files.download(f'img_{img_idx}.png')


Batch: 150
20/20 [==============================] - 29s 1s/step
Saving image: 750
Saving image: 751
Saving image: 752
Saving image: 753
Saving image: 754

Batch: 151
20/20 [==============================] - 29s 1s/step
Saving image: 755
Saving image: 756
Saving image: 757
Saving image: 758
Saving image: 759

Batch: 152
20/20 [==============================] - 28s 1s/step
Saving image: 760
Saving image: 761
Saving image: 762
Saving image: 763
Saving image: 764

Batch: 153
20/20 [==============================] - 29s 1s/step
Saving image: 765
Saving image: 766
Saving image: 767
Saving image: 768
Saving image: 769

Batch: 154
20/20 [==============================] - 29s 1s/step
Saving image: 770
Saving image: 771
Saving image: 772
Saving image: 773
Saving image: 774

Batch: 155
20/20 [==============================] - 27s 1s/step
Saving image: 775
Saving image: 776
Saving image: 777
Saving image: 778
Saving image: 779

Batch: 156
20/20 [==============================] - 27s 1s/step
Savin